Road space evolution
====================
Reconstructs the road space allocation in a given year

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import snman
from snman.constants import *

PERIMETER = 'cambridge_extended'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)

export_path = os.path.join(data_directory, 'roadspace_evolution', '_debug')

CRS_internal = 29119    # for Boston
#CRS_internal = 2056      # for Zurich

CRS_for_export = 4326

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [ ]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)

In [ ]:
lane_updates_cleaned_gpd = snman.io.import_geofile_to_gdf(
    os.path.join(export_path, 'lane_updates_cleaned.gpkg'),
    crs=CRS_internal
)

# keep only major cycling infra
# excluding shared streets and shared lane pavement markings
lane_updates_cleaned_gpd = lane_updates_cleaned_gpd.query('BIKE_FAC not in [2,6,7]')
lane_updates_cleaned_gpd

In [ ]:
#G = snman.street_graph.filter_lanes_by_modes(G, [MODE_PRIVATE_CARS, MODE_TRANSIT], delete_empty_edges=False)

In [ ]:
path = os.path.join(inputs_path, 'usa', 'cambridge', 'BikeFacilities_by_year', 'metadata', 'BIKE_FAC Lookup.csv')
bike_fac_names = pd.read_csv(path)

In [ ]:
# IMPORT CYCLING FACILITIES IN EACH YEAR AND CREATE THE LN_DESC KEY FOR EVERY YEAR
path = os.path.join(inputs_path, 'usa', 'cambridge', 'BikeFacilities_by_year', 'geojson_selected')
lane_updates = gpd.GeoDataFrame()

lane_keys = {}

for filename in os.listdir(path):
    
    file_path = os.path.join(path, filename)
    if not os.path.isfile(file_path):
        continue
    bike_facilities = snman.io.import_geofile_to_gdf(file_path, crs=CRS_internal)
    #bike_facilities = bike_facilities.query("BIKE_FAC in [1,5,4,8,9,11,12,41,42,81,82,83,84,92,91]")
    
    lane_update = gpd.GeoDataFrame(bike_facilities.geometry)
    # extract the year from the file name
    year = int(filename[14:18])
    lane_update['from_year'] = year
    lane_update['to_year'] = year
    lane_keys[year] = 'steps_' + str(year)
    lane_update['add_lanes'] = lane_update.apply(lambda row: 'L-', axis=1)
    lane_update['BIKE_FAC'] = bike_facilities['BIKE_FAC']
    lane_updates = pd.concat([lane_updates, lane_update], ignore_index=True)
    

# prepare the resulting lane updates
import shapely as shp
lane_updates.geometry = lane_updates.geometry.apply(
    lambda geom: shp.ops.linemerge(
        snman.geometry_tools.ensure_multilinestring(geom)
    )
)
gdf_lines = lane_updates.explode()
gdf_lines.reset_index(drop=True, inplace=True)

In [ ]:
lane_updates = pd.merge(lane_updates, bike_fac_names, on='BIKE_FAC', how='left').reset_index(drop=True)
lane_updates = gpd.GeoDataFrame(lane_updates)
snman.io.export_gdf(lane_updates, os.path.join(export_path, 'lane_updates.gpkg'))

In [ ]:
# MATCH THE LANE UPDATES ONTO THE STREET GRAPH

import copy
import geopandas as gpd
import shapely as shp

for year, lane_key in lane_keys.items():
    
    print(year)
    
    for uvk, data in G.edges.items():
        data[lane_key] = copy.deepcopy(data[KEY_LANES_DESCRIPTION])
    
    lanes = gpd.GeoDataFrame(lane_updates_cleaned_gpd.query(f"from_year <= {year} <= to_year"))
    
    lanes_opposite = copy.deepcopy(lanes)
    lanes_opposite = lanes_opposite.query('oneway == False')
    lanes_opposite.geometry = lanes_opposite.geometry.apply(
        lambda geom: shp.ops.substring(geom, 1, 0, normalized=True)
    )
    
    lanes = gpd.GeoDataFrame(pd.concat([lanes, lanes_opposite], ignore_index=True), crs=lanes.crs)

    """
    snman.enrichment.match_lane_edits(G, lanes,
        remove_short_overlaps=True,
        remove_sidetrips=False,
        max_dist=30, max_dist_init=30, max_lattice_width=5,
        lanes_key=lane_key
    )
    """
    
    snman.enrichment.match_linestrings(
        G, lanes,
        [{'source_column': 'BIKE_FAC', 'target_column': '_factype_' + lane_key, 'agg': 'list'}],
        remove_short_overlaps=True,
        remove_sidetrips=False,
        max_dist=400, max_dist_init=100, max_lattice_width=5,
        _save_map=(os.path.join(export_path, '_matching_edges.gpkg'), os.path.join(export_path, '_matching_nodes.gpkg'))
    )
    

In [ ]:
# CALCULATE CHANGE ON EDGES
if 1:
    for uvk, data in G.edges.items():
        
        previous_lane_key = None
        for year, lane_key in lane_keys.items():
            
            if previous_lane_key is None:
                data['_bike_fac_change_' + lane_key] = 0
                
            else:
            
                bike_facilities_previous_year = set(
                    data['_factype_' + previous_lane_key + '_forward'] + 
                    data['_factype_' + previous_lane_key + '_backward']
                )
                
                bike_facilities_this_year = set(
                    data['_factype_' + lane_key + '_forward'] + 
                    data['_factype_' + lane_key + '_backward']
                )
                
                if bike_facilities_this_year != bike_facilities_previous_year:
                    data['_bike_fac_change_' + lane_key] = 1
                else:
                    data['_bike_fac_change_' + lane_key] = 0
                
            previous_lane_key = lane_key
            
    edges_df = snman.oxc.graph_to_gdfs(G, nodes=False).assign(x=1)
    timeline_df = pd.DataFrame({'year': lane_keys.keys(), 'lanes_key': lane_keys.values()}).assign(x=1)
    
    changes_df = pd.merge(edges_df, timeline_df, on='x')
    
    changes_df['change'] = changes_df.apply(lambda row: row['_bike_fac_change_steps_' + str(row['year'])], axis=1)
    
    changes_df = changes_df.query('change==1')
    
    changes_df = gpd.GeoDataFrame(changes_df[['osmid', 'year', 'lanes_key', 'change', 'geometry']])
    
    snman.io.export_gdf(changes_df, os.path.join(export_path, 'edge_changes.gpkg'))

In [ ]:
# CATEGORIZE PROJECTS ACCORDING TO INTERVIEW MATRIX

In [ ]:
#CHECK IF LINKS ARE PART OF THE PLANNED NETWORK

# Load the Cycling Safety Ordinance (CSO) network
cso_gpkg = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'usa/cambridge/CSO/cso.gpkg'),
    crs=CRS_internal
)

# Match the SCO onto the street graph
snman.enrichment.match_linestrings(
    G, cso_gpkg,
    [{'source_column': 'type', 'target_column': '_cso_type', 'agg': 'list'}],
    remove_short_overlaps=True,
    remove_sidetrips=False,
    max_dist=400, max_dist_init=100, max_lattice_width=5
)

for uvk, data in G.edges.items():
    if 'mass_ave_4' in data['_cso_type_forward'] + data['_cso_type_backward']:
        data['cso_type'] = 'mass_ave_4'
    elif 'mass_ave_rest' in data['_cso_type_forward'] + data['_cso_type_backward']:
        data['cso_type'] = 'mass_ave_rest'
    elif 'special_4' in data['_cso_type_forward'] + data['_cso_type_backward']:
        data['cso_type'] = 'special_4'
    elif 'other' in data['_cso_type_forward'] + data['_cso_type_backward']:
        data['cso_type'] = 'other'
    else:
        data['cso_type'] = 'none'
    

In [ ]:
# CHECK IF SEPARATED INFRASTRUCTURE IS IMPLEMENTED
factypes_separated_bikelanes = {4,8,84,83}
for uvk, data in G.edges.items():
    # Check if separated bike lanes are implemented
    #separated_bikelanes_implemented = factypes_separated_bikelanes in data['_factype_steps_2023_forward']
    #print([data['_factype_' + key + '_forward'] for key in lane_keys])
    #print(data['_factype_steps_2023_forward'])
    results = []
    for key in ['_factype_steps_2023_forward', '_factype_steps_2023_backward']:
        facilities = data[key]
        # unstringify the list
        facilities = facilities.strip('][').split(',')
        facilities = [int(float(facility)) for facility in facilities if len(facility) > 0]
        print(facilities)
        direction_has_separated_bikelanes = len(set(facilities) & factypes_separated_bikelanes) > 0
        #print(set(facilities) & factypes_separated_bikelanes)
        results += [direction_has_separated_bikelanes]
        
    data['has_separated_bikelanes'] = any(results)
    if all(results):
        data['has_separated_bikelanes'] = 'both_directions'
    elif results[0] == True:
        data['has_separated_bikelanes'] = 'only_forward'
    elif results[1] == True:
        data['has_separated_bikelanes'] = 'only_backward'
    else:
        data['has_separated_bikelanes'] = 'no'
    
    

In [ ]:
if 0:
    def cycling_lanes_length(lanes_key):
        L = snman.lane_graph.create_lane_graph(H, lanes_key)
        return sum(
            [
                e['length']
                for uvk, e
                in L.edges.items()
                if e['lanetype'] == LANETYPE_CYCLING_LANE and e['width'] > 0
            ]
        ) / 1000
    
    def car_lanes_length(lanes_key):
        L = snman.lane_graph.create_lane_graph(H, lanes_key)
        return sum(
            [
                e['length']
                for uvk, e
                in L.edges.items()
                if e['lanetype'] == LANETYPE_MOTORIZED and e['width'] > 0
            ]
        ) / 1000

In [ ]:
# GENERATE METRICS
if 0:
    import pandas as pd
    import copy
    
    timeline_df = pd.DataFrame()
    
    for name, data in measurement_regions_gdf.iterrows():
        
        polygon = data.geometry
        H = snman.oxc.truncate.truncate_graph_polygon(G, polygon, quadrat_width=100, retain_all=True)
        
        timeline_df_one_region = pd.DataFrame({'year': lane_keys.keys(), 'lanes_key': lane_keys.values()})
        timeline_df_one_region['measurement_region_name'] = name
        timeline_df_one_region['cycling_lanes_length'] = timeline_df_one_region['lanes_key'].apply(cycling_lanes_length)
        timeline_df_one_region['change_metric'] = timeline_df_one_region['cycling_lanes_length'].diff().fillna(0)
        
        #print(timeline_df_one_region)
        
        timeline_df = pd.concat([timeline_df, timeline_df_one_region], axis=0, ignore_index=True)
        
        timeline_gdf = gpd.GeoDataFrame(
            pd.merge(timeline_df, measurement_regions_gdf, left_on='measurement_region_name', right_index=True, how='left')
        )  
        
        snman.io.export_gdf(timeline_gdf, os.path.join(export_path, 'timeline.gpkg'))

In [ ]:
import matplotlib.pyplot as plt
import textwrap
from matplotlib.lines import Line2D

projects_gdf['x'] = projects_gdf['year_from'] + 0.5

plt.rcParams['font.size'] = 8

fig, ax = plt.subplots(figsize=(20, 5))

ax.set_xlabel('Year')
ax.set_ylabel('Category')

ax.set_xlim([2004, 2029])
ax.set_ylim([0, 10])

for index, row in projects_gdf.iterrows():
    marker_width = row['length_m'] / 2500
    for direction in [0,1]:
        offset = 0.1 * direction
        ax.add_line(
            Line2D(
                [row['x'] - marker_width / 2, row['x'] + marker_width / 2],
                [row['y'] + offset, row['y'] + offset],
                color=row['bike_facility_color'],
                lw=1.5, label='One'
            )
        )
        if row['only_one_direction'] == True:
            break

for i, label in enumerate(projects_gdf['name']):
    ax.annotate('\n'.join(textwrap.wrap(textwrap.shorten(label, 30, placeholder='...'), width=13)), (projects_gdf['x'][i], projects_gdf['y'][i]+0), textcoords="offset points", xytext=(0,-3), ha='center', va='top', fontsize='6')

for index, row in grouped_df.iterrows():
    if index in {'none', 'other', 'mass_ave_rest', 'mass_ave_4'}:
        ax.axhline(y=row['y_min'], color='gray', linestyle=':', label='Threshold', linewidth=0.3)
    else:
        ax.axhline(y=row['y_min'], color='black', linestyle='--', label='Threshold', linewidth=0.5)

x_milestones = {2020: 'Amended CSO', 2024: 'Now'}

for x, name in x_milestones.items():
    ax.axvline(x=x, color='black', linestyle='--', linewidth=0.5)

ax.set_yticks(grouped_df['y_min'], grouped_df['fullname'].apply(lambda x: '\n'.join(textwrap.wrap(x, 10) + [' '])), va='bottom')  # Adjust the tick positions and labels
y_ticks = range(int(min(projects_gdf['year_from'])), 2025)
ax.set_xticks(y_ticks)

legend_elements = [
    Line2D([0], [0], color='b', lw=2, label='One'),
    Line2D([0], [0], color='r', lw=2, label='Tro')
]

legend_df = pd.DataFrame(projects_gdf.groupby('bike_facility').agg({'bike_facility_color': 'first', 'bike_facility_name': 'first'}))

legend_elements = [
    Line2D([0], [0], color=row['bike_facility_color'], lw=2, label=row['bike_facility_name'])
    for index, row
    in legend_df.iterrows()
]

ax.legend(handles=legend_elements, loc='upper left')

plt.savefig('scatter_plot_custom_dpi.png', dpi=300) 

In [ ]:
pd.DataFrame(projects_gdf.groupby('bike_facility').agg({'bike_facility_color': 'first', 'bike_facility_name': 'first'}).reset_index())

Export
------
Saves the resulting datasets to the disk. Use the *snman_detailed.qgz* file to view them in QGIS.

In [ ]:
if 1:
    print('Export network without lanes')
    snman.io.export_street_graph(
        G,
        os.path.join(export_path, 'street_graph_edges.gpkg'),
        os.path.join(export_path, 'street_graph_nodes.gpkg'),
        crs=CRS_for_export,
        lane_keys=[KEY_LANES_DESCRIPTION, KEY_GIVEN_LANES_DESCRIPTION, KEY_LANES_DESCRIPTION_AFTER] + list(lane_keys.values())
    )

if 0:
    print('Export lane geometries')
    SCALING = 1
    snman.io.export_street_graph_with_lanes(
        G,
        list(lane_keys.values()),
        os.path.join(export_path, 'lane_geometries.shp'),
        scaling=SCALING,
        crs=CRS_for_export
    )